In [813]:
import requests
import json
import pandas as pd
from datetime import datetime

### Set the time for Now

In [814]:
now = datetime.utcnow()

### Get bearer token

In [815]:
username = "timothy_woodland@outlook.com"
password = "@j-tLtrR8F"
client_secret = 'wZaRN7rpjn3FoNyF5IFuxg9uMzYJcvOoQ8QWiIqS3hfk6gLhVlG57j5YNoZL2Rtc'

In [816]:
payload = {'client_id':'tado-web-app',
            'grant_type':'password',
            'scope':'home.user',
            'username':username,
            'password':password,
            'client_secret':client_secret}

In [817]:
token_r = requests.post('https://auth.tado.com/oauth/token', params=payload)
token = token_r.json()["access_token"]
# print(token)

### Set Header for future Requests

In [818]:
header = {'Authorization' : "Bearer " + token}

### Get Home ID

In [819]:
home_id_r = requests.get("https://my.tado.com/api/v1/me", headers = header)
home_id = home_id_r.json()['homeId']
# print(home_id)

### Get name of home

In [820]:
home_url = "https://my.tado.com/api/v2/homes/"+str(home_id)
home_r = requests.get(home_url , headers = header)

In [821]:
home = home_r.json()["name"]

In [822]:
print(home)

Leslie Road


### Get zone data

In [823]:
zones_list_url = home_url+"/zones"

In [824]:
zone_list_r = requests.get(zones_list_url , headers = header)

In [825]:
zone_list_json = zone_list_r.json()
# print(json.dumps(zone_list_json, indent=1))

In [826]:
zone_list = []
zone_dict = {}
counter = 1


for zone_data in zone_list_json:
    zone_id = zone_data["id"]
    zone_name = zone_data["name"]

    zone_list.append(zone_id)
    
    zone_dict[zone_id] = [now, zone_name]

    counter = counter + 1


In [827]:
for zone in zone_list:
    url = url+"/"+str(zone)+"/state"
    # header = {'Authorization' : "Bearer " + token}
    

### Get data from a Zone

In [828]:
zone = 1

In [829]:
zone_url = zones_list_url+"/"+str(zone)+"/state"

In [830]:
zone_r = requests.get(zone_url , headers = header)

In [831]:
zone_json = zone_r.json()

In [832]:
print(zone_json)

{'tadoMode': 'HOME', 'geolocationOverride': False, 'geolocationOverrideDisableTime': None, 'preparation': None, 'setting': {'type': 'HEATING', 'power': 'ON', 'temperature': {'celsius': 20.0, 'fahrenheit': 68.0}}, 'overlayType': None, 'overlay': None, 'openWindow': None, 'nextScheduleChange': {'start': '2023-12-16T17:00:00Z', 'setting': {'type': 'HEATING', 'power': 'ON', 'temperature': {'celsius': 21.0, 'fahrenheit': 69.8}}}, 'nextTimeBlock': {'start': '2023-12-16T17:00:00.000Z'}, 'link': {'state': 'ONLINE'}, 'activityDataPoints': {'heatingPower': {'type': 'PERCENTAGE', 'percentage': 0.0, 'timestamp': '2023-12-16T10:16:21.718Z'}}, 'sensorDataPoints': {'insideTemperature': {'celsius': 20.03, 'fahrenheit': 68.05, 'timestamp': '2023-12-16T10:12:33.091Z', 'type': 'TEMPERATURE', 'precision': {'celsius': 0.1, 'fahrenheit': 0.1}}, 'humidity': {'type': 'PERCENTAGE', 'percentage': 60.5, 'timestamp': '2023-12-16T10:12:33.091Z'}}}


In [833]:
mode = zone_json["tadoMode"]
geo_override = zone_json["geolocationOverride"]
type = zone_json["setting"]["type"]
status = zone_json["setting"]["power"]
set_temp = zone_json["setting"]["temperature"]["celsius"]
power_level = zone_json["activityDataPoints"]["heatingPower"]["percentage"]
zone_temp = zone_json["sensorDataPoints"]["insideTemperature"]["celsius"]
zone_temp = zone_json["sensorDataPoints"]["humidity"]["percentage"]

In [834]:
zone_temp = zone_json["sensorDataPoints"]["humidity"]["percentage"]

In [835]:
print(json.dumps(zone_r.json(), indent=2))

{
  "tadoMode": "HOME",
  "geolocationOverride": false,
  "geolocationOverrideDisableTime": null,
  "preparation": null,
  "setting": {
    "type": "HEATING",
    "power": "ON",
    "temperature": {
      "celsius": 20.0,
      "fahrenheit": 68.0
    }
  },
  "overlayType": null,
  "overlay": null,
  "openWindow": null,
  "nextScheduleChange": {
    "start": "2023-12-16T17:00:00Z",
    "setting": {
      "type": "HEATING",
      "power": "ON",
      "temperature": {
        "celsius": 21.0,
        "fahrenheit": 69.8
      }
    }
  },
  "nextTimeBlock": {
    "start": "2023-12-16T17:00:00.000Z"
  },
  "link": {
    "state": "ONLINE"
  },
  "activityDataPoints": {
    "heatingPower": {
      "type": "PERCENTAGE",
      "percentage": 0.0,
      "timestamp": "2023-12-16T10:16:21.718Z"
    }
  },
  "sensorDataPoints": {
    "insideTemperature": {
      "celsius": 20.03,
      "fahrenheit": 68.05,
      "timestamp": "2023-12-16T10:12:33.091Z",
      "type": "TEMPERATURE",
      "precision"

In [836]:
json_test = zone_r.json()

In [837]:
json_test["sensorDataPoints"]["insideTemperature"]["celsius"]

20.03

In [838]:
json_test["sensorDataPoints"]["humidity"]["percentage"]

60.5

### Test Weather

In [839]:
weather_url = "https://my.tado.com/api/v2/homes/"+str(home_id)+"/weather"

In [840]:
r = requests.get(weather_url , headers = header)

In [841]:
# print(json.dumps(r.json(), indent=2))

In [842]:
test_json = r.json()

In [843]:
json_df = pd.DataFrame.from_dict(test_json)

In [845]:
test_json["weatherState"]["value"]

'CLOUDY'

In [846]:
test_json["outsideTemperature"]["celsius"]

10.03

### Get Weather and Build df

In [847]:
weather_json = r.json()

In [848]:
weather_data = [weather_json["outsideTemperature"]["celsius"], weather_json["weatherState"]["value"], "tado"]

In [849]:
weather_columns = ["temperature_C", "state", "source"]

In [850]:
weather_df = pd.DataFrame(data=[weather_data], index=[now], columns=weather_columns)